# Импорты

In [1]:
!pip install transformers datasets trl peft accelerate bitsandbytes torch huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer, SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from tqdm.notebook import tqdm
import pandas as pd
import torch
import ast

# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


In [3]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
dataset_name = "notbadai/python_functions_reasoning"

кажется, с моделькой беды, всё равно сдам

# Загрузка модели

In [4]:
device = 'cuda'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

default_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
default_model = prepare_model_for_kbit_training(default_model)
default_model = get_peft_model(default_model, lora_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

# Промпт и форматирование

In [7]:
system_prompt = """
You are an assistant who writes correct and working Python code.For each request (function description), you must return only one Python function.The response should be formatted as:\n``python\n# is your code\n``Don't write any explanations. Only the code in the markdown block. The code must be valid and match the request.
"""

def chatml_format(example):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": example['prompt']},
        {"role": "assistant", "content": example['answer']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {
        "prompt": prompt,
        "completion": example['answer']
    }


# Загрузка датасета

In [8]:
dataset = load_dataset(dataset_name)['train'].shuffle(seed=42)

sft_dataset = dataset.select(range(100, 200)).map(chatml_format)
grpo_dataset = dataset.select(range(0, 100)).map(chatml_format)
test_dataset = dataset.select(range(200, 300)).map(chatml_format)

README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/414M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/206299 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# SFT

In [9]:
training_args = SFTConfig(
    dataset_text_field="text",
    max_length=1024,
    report_to="none",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


In [10]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=sft_dataset,
    args=training_args,
)

Adding EOS to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.069900
20,0.060900
30,0.050000
40,0.037000
50,0.035400
60,0.034900
70,0.030400


TrainOutput(global_step=75, training_loss=0.04453075110912323, metrics={'train_runtime': 191.5417, 'train_samples_per_second': 1.566, 'train_steps_per_second': 0.392, 'total_flos': 223938931569408.0, 'train_loss': 0.04453075110912323})

# GRPO

In [12]:
def reward_format(prompts=None, completions=None, completion_ids=None, **kwargs):
    rewards = []
    for completion in completions:
        text = completion.strip()
        reward = 1.0 if text.startswith("```python") and text.endswith("```") else 0.0
        rewards.append(reward)
    return torch.tensor(rewards)

def reward_code_validity(prompts=None, completions=None, completion_ids=None, **kwargs):
    rewards = []
    for completion in completions:
        try:
            code = completion.strip()
            if code.startswith("```python"):
                code = code[len("```python"):]
            if code.endswith("```"):
                code = code[:-3]
            code = code.strip()
            ast.parse(code)
            rewards.append(1.0)
        except Exception:
            rewards.append(0.0)
    return torch.tensor(rewards)

In [13]:
training_args = GRPOConfig(
    max_completion_length=1024,
    logging_steps=10,
    report_to="none",
    per_device_train_batch_size=2,
    num_generations=2,
    num_train_epochs=1,
    label_names=["labels"]
)

In [14]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[reward_format, reward_code_validity],
    args=training_args,
    train_dataset=grpo_dataset,
    peft_config=lora_config
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'Qwen/Qwen2-0.5B-Instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [15]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.105800
20,-0.120200
30,-0.037000
40,0.113700
50,0.031300
60,0.141900
70,-0.052500
80,-0.016100
90,0.146900
100,-0.072000


TrainOutput(global_step=100, training_loss=0.024194785803556444, metrics={'train_runtime': 2055.9435, 'train_samples_per_second': 0.049, 'train_steps_per_second': 0.049, 'total_flos': 0.0, 'train_loss': 0.024194785803556444})

# Оценка

In [20]:
def evaluate(model, tokenizer, dataset, n=100):
    model.eval()
    format_scores = []
    valid_scores = []
    for example in tqdm(dataset.select(range(n))):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": example["prompt"]},
        ]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=200)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        format_scores.append(reward_format(completions=[response]).item())
        valid_scores.append(reward_code_validity(completions=[response]).item())

    avg_reward_format = sum(format_scores) / len(format_scores)
    avg_reward_valid = sum(valid_scores) / len(valid_scores)
    print(f"Average format reward: {avg_reward_format:.2f}")
    print(f"Average code validity reward: {avg_reward_valid:.2f}")

In [21]:
print("Original model:")
evaluate(default_model, tokenizer, test_dataset)

Original model:


  0%|          | 0/100 [00:00<?, ?it/s]

Average format reward: 0.00
Average code validity reward: 0.00


In [22]:
print("GPRO model:")
evaluate(model, tokenizer, test_dataset)

GPRO model:


  0%|          | 0/100 [00:00<?, ?it/s]

Average format reward: 0.00
Average code validity reward: 0.00
